# 从DataBase里面读取数据

In [2]:
time_start = 20150101
time_end =  20210630
feat_list = ['pre_close','open','high','low','close','vwap','volume','pct_chg','turnover','free_turnover','chg']
return_value = '5d_ret'
Alpha_Name = "AlphaNet_Original_Input_12.14"
output_path = '/home/wuwenjun/Data/' + Alpha_Name +'/'

In [3]:
import time
import os
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm

# New

In [13]:
DataAPI.get_stock_daily_return(date=pd.to_datetime("20150116"),ti =  0, tp = 0, price='close',
                           period = "5d")

5d_ret  trading_status
timestamp           ticker                          
2015-01-16 09:30:00 000001 -0.067532               0
                    000002 -0.014524               0
                    000004  0.074488               0
                    000005  0.000000               3
                    000006 -0.032159               0
...                              ...             ...
                    603806  0.072989               0
                    603889  0.089091               0
                    603988  0.194118               0
                    603993  0.148936               0
                    603998  0.126154               0

[2592 rows x 2 columns]

In [4]:
data = pd.read_csv("/home/wuwenjun/Data/AlphaNet_Original_Input.csv")
data

,timestamp,ticker,pre_close,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,chg,5d_ret
0,2015-01-06 09:30:00,1,15.84,15.99,16.28,15.60,16.02,15.9605,2860436.43,1.1364,2.9079,5.7657,0.18,-0.075710
1,2015-01-06 09:30:00,2,13.90,14.39,15.29,14.22,14.91,14.7858,6560835.70,7.2662,6.7834,8.1741,1.01,-0.106164
2,2015-01-06 09:30:00,4,15.57,15.58,15.90,15.30,15.69,15.5709,28095.12,0.7707,3.3496,4.5343,0.12,0.006390
3,2015-01-06 09:30:00,6,7.05,7.10,7.39,7.02,7.08,7.1881,1078647.12,0.4255,8.0562,13.2751,0.03,-0.078571
4,2015-01-06 09:30:00,7,14.60,14.51,14.80,14.25,14.52,14.5105,28415.72,-0.5479,1.3807,1.7538,-0.08,-0.004844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4851233,2021-07-01 09:30:00,688699,237.77,285.32,285.32,285.32,285.32,285.3200,1225.82,19.9983,0.6940,0.6940,47.55,-0.024476
4851234,2021-07-01 09:30:00,688777,90.00,89.95,94.80,89.06,94.15,92.9798,19001.61,4.6111,4.6288,4.6288,4.15,0.053133
4851235,2021-07-01 09:30:00,688788,138.20,137.28,148.50,136.33,145.38,144.3683,13572.56,5.1954,7.7977,7.7977,7.18,-0.102721
4851236,2021-07-01 09:30:00,688819,43.68,43.40,44.98,43.31,44.49,44.3727,47249.01,1.8544,4.6754,4.6754,0.81,0.032164


In [9]:
DataAPI.get_stock_daily_return(date=time_list,ti =  0, tp = 0, price='close',
                           period = "5d")

ValueError: time data "[Timestamp('2015-01-06 09:30:00'), Timestamp('2015-01-07 09:30:00'), Timestamp('2015-01-08 09:30:00'), Timestamp('2015-01-09 09:30:00'), Timestamp('2015-01-12 09:30:00'), Timestamp('2015-01-13 09:30:00'), Timestamp('2015-01-14 09:30:00'), Timestamp('2015-01-15 09:30:00'), Timestamp('2015-01-16 09:30:00'), Timestamp('2015-01-19 09:30:00'), Timestamp('2015-01-20 09:30:00'), Timestamp('2015-01-21 09:30:00'), Timestamp('2015-01-22 09:30:00'), Timestamp('2015-01-23 09:30:00'), Timestamp('2015-01-26 09:30:00'), Timestamp('2015-01-27 09:30:00'), Timestamp('2015-01-28 09:30:00'), Timestamp('2015-01-29 09:30:00'), Timestamp('2015-01-30 09:30:00'), Timestamp('2015-02-02 09:30:00'), Timestamp('2015-02-03 09:30:00'), Timestamp('2015-02-04 09:30:00'), Timestamp('2015-02-05 09:30:00'), Timestamp('2015-02-06 09:30:00'), Timestamp('2015-02-09 09:30:00'), Timestamp('2015-02-10 09:30:00'), Timestamp('2015-02-11 09:30:00'), Timestamp('2015-02-12 09:30:00'), Timestamp('2015-02-13 09

In [12]:
return_data = pd.DataFrame()
for i in tqdm(time_list):
    return_data = pd.concat([return_data,DataAPI.get_stock_daily_return(date=i,ti =  0, tp = 0, price='close',
                           period = "5d")],axis=0)
return_data

100%|██████████| 1580/1580 [13:02<00:00,  2.02it/s]


5d_ret  trading_status
timestamp           ticker                          
2015-01-06 09:30:00 000001 -0.075710               0
                    000002 -0.106164               0
                    000004  0.006390               0
                    000005  0.000000               3
                    000006 -0.078571               0
...                              ...             ...
2021-07-01 09:30:00 688700  0.153171               0
                    688777  0.053133               0
                    688788 -0.102721               0
                    688819  0.032164               0
                    688981 -0.058075               0

[5370219 rows x 2 columns]

In [25]:
data = pd.merge(data,return_data,left_index=True,right_index=True,how = "left")

In [32]:
data.to_csv(output_path + "data.csv")

# Original

In [13]:
dataframe_list = DataAPI.get_stock_prices(start_date=time_start, end_date=time_end).reset_index().set_index(['timestamp','ticker'])

Process ForkPoolWorker-60:
Process ForkPoolWorker-59:
Process ForkPoolWorker-55:
Process ForkPoolWorker-56:
Process ForkPoolWorker-57:
Process ForkPoolWorker-54:
Process ForkPoolWorker-58:
Process ForkPoolWorker-33:
Process ForkPoolWorker-27:
Process ForkPoolWorker-22:
Process ForkPoolWorker-32:
Process ForkPoolWorker-34:
Process ForkPoolWorker-53:
Process ForkPoolWorker-8:
Process ForkPoolWorker-70:
Process ForkPoolWorker-65:
Process ForkPoolWorker-69:
Process ForkPoolWorker-66:
Process ForkPoolWorker-5:
Process ForkPoolWorker-63:
Process ForkPoolWorker-67:
Process ForkPoolWorker-68:
Process ForkPoolWorker-72:
Process ForkPoolWorker-7:
Process ForkPoolWorker-16:
Process ForkPoolWorker-43:
Process ForkPoolWorker-71:
Process ForkPoolWorker-11:
Process ForkPoolWorker-10:
Process ForkPoolWorker-9:
Process ForkPoolWorker-14:
Process ForkPoolWorker-44:
Process ForkPoolWorker-6:
Process ForkPoolWorker-21:
Process ForkPoolWorker-1:
Process ForkPoolWorker-19:
Process ForkPoolWorker-17:
Process

KeyboardInterrupt: 

  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  Fil

  File "/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/io/parquet.py", line 218, in read
    mode="rb",
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/io/parquet.py", line 77, in _get_path_or_handle
    and not os.path.isdir(path_or_handle)
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/io/parquet.py", line 77, in _get_path_or_handle
    and not os.path.isdir(path_or_handle)
  File "/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/io/parquet.py", line 77, in _get_path_or_handle
    and not os.path.isdir(path_or_handle)
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, 

  File "/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/io/parquet.py", line 460, in read_parquet
    path, columns=columns, use_nullable_dtypes=use_nullable_dtypes, **kwargs
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
KeyboardInterrupt
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/io/parquet.py", line 218, in read
    mode="rb",
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/io/parquet.py", li

...

  File "/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/io/parquet.py", line 218, in read
    mode="rb",
  File "/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/io/parquet.py", line 218, in read
    mode="rb",
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/usr/local/miniconda/lib/python3.7/genericpath.py", line 42, in isdir
    st = os.stat(s)
Traceback (most recent call last):
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/usr/local/miniconda/lib/python3.7/genericpath.py", line 42, in isdir
    st = os.stat(s)
  File "/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/io/parquet.py", line 77, in _get_path_or_handle
    and not os.path.isdir(path_or_handle)
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/w

  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/io/parquet.py", line 460, in read_parquet
    path, columns=columns, use_nullable_dtypes=use_nullable_dtypes, **kwargs
  File "/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/io/parquet.py", line 460, in read_parquet
    path, columns=columns, use_nullable_dtypes=use_nullable_dtypes, **kwargs
  File "/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/io/parquet.py", line 460, in read_parquet
    path, columns=columns, use_nullable_dtypes=use_nullable_dtypes, **kwargs
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/io/parqu

Traceback (most recent call last):
  File "/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/io/parquet.py", line 218, in read
    mode="rb",
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ShareFolder/lgc/Modules/DataAPI/StockApi.py", line 335, in _get_stock_daily_data
    df = pd.read_parquet(path)
  File "/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/io/parquet.py", line 77, in _get_path_or_handle
    and not os.path.isdir(path_or_handle)
  File "/home/ShareFolder/lgc/Modules/DataAPI/StockApi.py", line 335, in _get_stock_daily_data
    df = pd.read_parquet(path)
  File "/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/io/parquet.py", line 460, in read_parquet
    path, columns=columns, use_nullable_dtypes=use_nullable_dtypes, **kwargs
  File "/usr/local/miniconda/lib/python3.7/genericpath.py", line 42, in isdir
    st = os.stat(s)
  File "/usr/local/miniconda

  File "/home/ShareFolder/lgc/Modules/DataAPI/StockApi.py", line 335, in _get_stock_daily_data
    df = pd.read_parquet(path)
  File "/usr/local/miniconda/lib/python3.7/genericpath.py", line 42, in isdir
    st = os.stat(s)
  File "/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/io/parquet.py", line 218, in read
    mode="rb",
  File "/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/io/parquet.py", line 218, in read
    mode="rb",
  File "/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/io/parquet.py", line 460, in read_parquet
    path, columns=columns, use_nullable_dtypes=use_nullable_dtypes, **kwargs
KeyboardInterrupt
  File "/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/io/parquet.py", line 218, in read
    mode="rb",
  File "/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/io/parquet.py", line 218, in read
    mode="rb",
  File "/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/io/parquet.py", line 460, in read_parquet
    path, c

  File "/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/io/parquet.py", line 77, in _get_path_or_handle
    and not os.path.isdir(path_or_handle)
  File "/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/io/parquet.py", line 218, in read
    mode="rb",
  File "/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/io/parquet.py", line 77, in _get_path_or_handle
    and not os.path.isdir(path_or_handle)
  File "/usr/local/miniconda/lib/python3.7/genericpath.py", line 42, in isdir
    st = os.stat(s)
KeyboardInterrupt
  File "/usr/local/miniconda/lib/python3.7/genericpath.py", line 42, in isdir
    st = os.stat(s)
  File "/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/io/parquet.py", line 77, in _get_path_or_handle
    and not os.path.isdir(path_or_handle)
  File "/home/wuwenjun/.local/lib/python3.7/site-packages/pandas/io/parquet.py", line 77, in _get_path_or_handle
    and not os.path.isdir(path_or_handle)
  File "/usr/local/miniconda/lib/python3.7/genericpa

In [ ]:
dataframe_list = dataframe_list[feat_list].reset_index()
time_df = pd.DataFrame(dataframe_list['timestamp'].drop_duplicates())
time_df['timestamp_shift'] = time_df['timestamp'].apply(lambda x : DataAPI.get_next_trading_day(x, output='datetime'))
dataframe_list = pd.merge(left = dataframe_list , right = time_df , how = 'left',on = 'timestamp')
dataframe_list.drop(['timestamp'],axis=1,inplace=True)
dataframe_list.rename(columns = {"timestamp_shift":"timestamp"},inplace=True)
dataframe_list['timestamp'] = pd.to_datetime(dataframe_list['timestamp']) + pd.Timedelta(hours=9,minutes = 30)
dataframe_list.set_index(['timestamp','ticker'],inplace = True)
dataframe_list

In [5]:
config_path = r'/home/ShareFolder/lgc/Modules/Research/config/feature_bt_template'
print('Loading the configuration from ' + config_path)
configs = namespace.load_namespace(config_path)
FT = FeatureAnalysis(configs, feature_path=r"/home/ShareFolder/factor_lib")

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


Process ForkPoolWorker-131:
Process ForkPoolWorker-102:
Process ForkPoolWorker-124:
Process ForkPoolWorker-135:
Process ForkPoolWorker-128:
Process ForkPoolWorker-142:
Process ForkPoolWorker-136:
Process ForkPoolWorker-140:
Process ForkPoolWorker-134:
Process ForkPoolWorker-93:
Process ForkPoolWorker-126:
Process ForkPoolWorker-123:
Process ForkPoolWorker-139:
Process ForkPoolWorker-104:
Process ForkPoolWorker-137:
Process ForkPoolWorker-101:
Process ForkPoolWorker-94:
Process ForkPoolWorker-122:
Process ForkPoolWorker-133:
Process ForkPoolWorker-141:
Process ForkPoolWorker-92:
Process ForkPoolWorker-120:
Process ForkPoolWorker-125:
Process ForkPoolWorker-130:
Process ForkPoolWorker-86:
Process ForkPoolWorker-83:
Process ForkPoolWorker-107:
Process ForkPoolWorker-129:
Process ForkPoolWorker-95:
Process ForkPoolWorker-88:
Process ForkPoolWorker-105:
Process ForkPoolWorker-138:
Process ForkPoolWorker-108:
Process ForkPoolWorker-99:
Process ForkPoolWorker-87:
Process ForkPoolWorker-144:
P

  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Process ForkPoolWorker-115:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.p

  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
Traceback (most recent call last):
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessi

  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run

  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/minico

KeyboardInterrupt
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
KeyboardInterrupt
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/local/miniconda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._sem

In [6]:
FT.load_feature_from_dataframe(dataframe_list,universe='Investable')
FT.load_return_data()

Merging feature and return...: 100%|█████████████████████████████████████████████████████████████████████████████| 1580/1580 [00:07<00:00, 215.63it/s]


In [7]:
data_info = FT.data_info

In [8]:
# Drop 掉不需要的return 列
df = pd.DataFrame()
return_list = feat_list.copy()
return_list.append(return_value)
for timestamp in tqdm(data_info.keys()):
    timestamp = pd.to_datetime(timestamp)
    one_data = data_info[timestamp][return_list].reset_index()
    one_data['timestamp'] = timestamp
    df = pd.concat([df,one_data],axis=0)
df.sort_values(by = ['timestamp','ticker'],inplace=True)
display(df)

100%|██████████| 1580/1580 [05:06<00:00,  5.15it/s]


,ticker,pre_close,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,chg,5d_ret,timestamp
0,000001,15.84,15.99,16.28,15.60,16.02,15.9605,2860436.43,1.1364,2.9079,5.7657,0.18,-0.075710,2015-01-06 09:30:00
1,000002,13.90,14.39,15.29,14.22,14.91,14.7858,6560835.70,7.2662,6.7834,8.1741,1.01,-0.106164,2015-01-06 09:30:00
2,000004,15.57,15.58,15.90,15.30,15.69,15.5709,28095.12,0.7707,3.3496,4.5343,0.12,0.006390,2015-01-06 09:30:00
3,000006,7.05,7.10,7.39,7.02,7.08,7.1881,1078647.12,0.4255,8.0562,13.2751,0.03,-0.078571,2015-01-06 09:30:00
4,000007,14.60,14.51,14.80,14.25,14.52,14.5105,28415.72,-0.5479,1.3807,1.7538,-0.08,-0.004844,2015-01-06 09:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4024,688699,237.77,285.32,285.32,285.32,285.32,285.3200,1225.82,19.9983,0.6940,0.6940,47.55,-0.024476,2021-07-01 09:30:00
4025,688777,90.00,89.95,94.80,89.06,94.15,92.9798,19001.61,4.6111,4.6288,4.6288,4.15,0.053133,2021-07-01 09:30:00
4026,688788,138.20,137.28,148.50,136.33,145.38,144.3683,13572.56,5.1954,7.7977,7.7977,7.18,-0.102721,2021-07-01 09:30:00
4027,688819,43.68,43.40,44.98,43.31,44.49,44.3727,47249.01,1.8544,4.6754,4.6754,0.81,0.032164,2021-07-01 09:30:00


In [ ]:
df.set_index(['timestamp','ticker']).to_csv(output_path + 'data.csv')
print(output_path + 'data.csv')